In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from urllib import request
from tabulate import tabulate
import pandas as pd
import requests
import random
import luigi
import time
import json
import csv 
import re
import os

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# URL produk Tokopedia
pages = ['https://www.tokopedia.com/exsportstore/exsport-scholar-backpack-two-tone-light-pink-1731047930520110328']

options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)

stock_data = []

try:
    for page in pages:  # Iterasi setiap link produk
        driver.get(page)  # Langsung akses URL

        # Menunggu hingga elemen body muncul
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )

        # Scroll untuk memuat halaman
        for _ in range(5):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
            time.sleep(2)

        # Mencari elemen utama produk
        try:
            container = driver.find_element(By.CLASS_NAME, "css-856ghu")
        except:
            print("Elemen utama tidak ditemukan")
            continue  # Skip ke URL berikutnya jika tidak ada produk

        # Mengambil Nama Produk
        try:
            name = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDetailProductName']").text.strip()
        except:
            name = None

        # Mengambil Stok Produk
        try:
            stock_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='stock-label']")
            stock_text = stock_elem.text.strip()
            stock = stock_text.split(":")[-1].strip() if "Stok Total" in stock_text else stock_text
        except:
            stock = None

        # Mengambil Kategori (Etalase)
        try:
            etalase_elem = container.find_element(By.CSS_SELECTOR, "li[class*='css-'] a b")
            etalase = etalase_elem.text.strip()
        except:
            etalase = None

        # Mengambil Deskripsi Produk
        try:
            description_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDescriptionProduk']")
            text_description = description_elem.text.strip()
        except:
            text_description = None

        # Menambahkan data ke list
        stock_data.append({
            'name_product': name,
            'stock': stock,
            'kategori': etalase,
            'description': text_description
        })

    # Mengonversi ke DataFrame
    exsport_stock_tokped_df = pd.DataFrame(stock_data)

    # Menyimpan ke CSV
    exsport_stock_tokped_df.to_csv('exsport_stock_tokped.csv', index=False)
    print("Data berhasil disimpan ke CSV!")

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

finally:
    driver.quit()  # Menutup browser


Data berhasil disimpan ke CSV!


In [ ]:
class ExtractTokpedExsportData(luigi.Task):
    def requires(self):
        pass # Tidak ada task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('extract-raw-data/exsport_tokped_raw.csv') # MTempat penyimpanan data yang diekstrak


    def run(self):
        base_url = "https://www.tokopedia.com/exsportstore/product/page/{}" # URL dasar untuk mengambil data produk exsport dari Tokopedia

        # Mengatur opsi untuk webdriver Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled') # Menonaktifkan fitur otomatisasi
        options.add_experimental_option('useAutomationExtension', False) # Menonaktifkan ekstensi otomatisasi
        options.add_experimental_option("excludeSwitches", ["enable-automation"]) # Mengecualikan switch otomatisasi
        driver = webdriver.Chrome(options=options) # Membuat instance dari webdriver Chrome

        product_data = [] # List untuk menyimpan data produk

        try:
            for page in range(1, 12): # Mengambil data dari halaman 1 hingga 11
                url = base_url.format(page) # Membuat URL untuk halaman saat ini
                driver.get(url) # Mengakses URL

                # Menunggu hingga elemen body muncul
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'body'))
                )

                # Menggulir halaman untuk memuat lebih banyak produk
                for _ in range(5):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Menggulir ke bawah
                    time.sleep(2) # Menunggu 2 detik
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);") # Menggulir ke atas
                    time.sleep(2) # Menunggu 2 detik

                # Mengambil elemen produk
                product_containers = driver.find_elements(By.CSS_SELECTOR, "[data-testid='divProductWrapper']")

                for container in product_containers: # Iterasi setiap elemen produk
                    try:
                        name = container.find_element(By.CSS_SELECTOR, "[data-testid='linkProductName']").text # Mengambil nama produk
                    except:
                        name = None # Jika gagal, set nama menjadi None
                    
                    # Mengambil link produk dari elemen
                    try:
                        link = container.find_element(By.CSS_SELECTOR, "a.pcv3__info-content").get_attribute('href') # Mencari elemen link produk dan mengambil atribut 'href'
                    except:
                        link = None # Jika gagal, set link menjadi None

                    # Mengambil harga jual produk dari elemen
                    try:
                        price_sale_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='linkProductPrice']") # Mencari elemen harga jual produk
                        price_sale = price_sale_elem.text if price_sale_elem else None # Mengambil teks dari elemen harga jual produk
                    except:
                        price_sale = None # Jika gagal, set harga jual menjadi None

                    # Mengambil harga asli produk dari elemen
                    try:
                        price_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblProductSlashPrice']") # Mencari elemen harga asli produk
                        price = price_elem.text if price_elem else None # Mengambil teks dari elemen harga asli produk
                    except:
                        price = None # Jika gagal, set harga asli menjadi None

                    try:
                        discount_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblProductDiscount']") # Mencari elemen diskon produk
                        discount = discount_elem.text if discount_elem else None # Mengambil teks dari elemen diskon produk
                    except:
                        discount = None # Jika gagal, set diskon menjadi None

                    # Mengambil rating produk dari elemen
                    try:
                        rating_elem = container.find_element(By.CSS_SELECTOR, ".prd_rating-average-text") # Mencari elemen rating produk
                        rating = rating_elem.text if rating_elem else None # Mengambil teks dari elemen rating produk
                    except:
                        rating = None # Jika gagal, set rating menjadi None
                    
                    # Mengambil jumlah produk yang terjual dari elemen
                    try:
                        sold_elem = container.find_element(By.CSS_SELECTOR, ".prd_label-integrity") # Mencari elemen jumlah produk yang terjual
                        sold = sold_elem.text if sold_elem else None # Mengambil teks dari elemen jumlah produk yang terjual
                    except:
                        sold = None # Jika gagal, set jumlah produk yang terjual menjadi None

                    # Mengambil link gambar produk dari elemen
                    try:
                        image_elem = container.find_element(By.CSS_SELECTOR, ".css-1q90pod") # Mencari elemen gambar produk
                        image = image_elem.get_attribute('src') if image_elem else None # Mengambil atribut 'src' dari elemen gambar produk
                    except:
                        image = None # Jika gagal, set link gambar menjadi None

                    # Menambahkan data produk ke dalam list product_data
                    product_data.append({
                        'name_product': name, # Nama produk
                        'product_link': link, # Link produk
                        'price_sale': price_sale, # Harga jual
                        'price_original': price, # Harga asli
                        'discount': discount, # Diskon
                        'sold': sold, # Jumlah produk yang terjual
                        'rating': rating, # Rating produk
                        'image_link': image # Link gambar produk
                    })

            # Mengonversi list product_data ke dalam DataFrame
            exsport_tokped_df = pd.DataFrame(product_data)

            # Menyimpan DataFrame ke dalam file CSV
            exsport_tokped_df.to_csv(self.output().path, index=False)

        except Exception as e:
            print(f"Terjadi kesalahan: {e}") # Menampilkan pesan kesalahan jika terjadi kesalahan
        
        finally:
            driver.quit() # Menutup browser

❌ Elemen utama produk tidak ditemukan.
✅ Data berhasil disimpan ke CSV!


In [14]:
luigi.build([ExtractTokpedExsportData()], local_scheduler=True) # Menjalankan task ExtractTokpedExsportData

DEBUG: Checking if ExtractTokpedExsportData() is complete
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 26588] Worker Worker(salt=3377661383, workers=1, host=zueible, username=LENOVO, pid=26588) running   ExtractTokpedExsportData()
INFO: [pid 26588] Worker Worker(salt=3377661383, workers=1, host=zueible, username=LENOVO, pid=26588) done      ExtractTokpedExsportData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=3377661383, workers=1, host=zueible, username=LENOVO, pid=26588) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 t

True

In [29]:
class ExtractTokpedStockExsportData(luigi.Task):
    def requires(self):
        return ExtractTokpedExsportData() # Task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('extract-raw-data/exsport_stock_tokped_raw.csv') # Tempat penyimpanan data yang diekstrak


    def run(self):
        extract_data = pd.read_csv(self.input().path) # Membaca file CSV yang diekstrak sebelumnya
        pages = extract_data['product_link'].tolist() # Mengambil kolom product_link dan mengonversinya ke dalam list

        # Mengatur opsi untuk webdriver Chrome
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        driver = webdriver.Chrome(options=options) # Membuat instance dari webdriver Chrome

        product_data = [] # List untuk menyimpan data produk

        try:
            for page in pages: # Mengambil data dari halaman 1 hingga 11
                url = page # Membuat URL untuk halaman saat ini
                driver.get(url) # Mengakses URL

                # Menunggu hingga elemen body muncul
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'body'))
                )

                # Menggulir halaman untuk memuat lebih banyak produk
                for _ in range(5):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Menggulir ke bawah
                    time.sleep(2) # Menunggu 2 detik
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);") # Menggulir ke atas
                    time.sleep(2) # Menunggu 2 detik

                # Mengambil elemen produk
                product_containers = driver.find_elements(By.CSS_SELECTOR, "[class='css-856ghu']")

                for container in product_containers: # Iterasi setiap elemen produk
                    try:
                        name = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDetailProductName']").text # Mengambil nama produk
                    except:
                        name = None # Jika gagal, set nama menjadi None

                    # Mengambil stock jual produk dari elemen
                    try:
                        stock_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='stock-label']")  # Mencari elemen stok
                        stock_text = stock_elem.text.strip()  # Mengambil teks dan menghapus spasi ekstra
                        
                        # Mengambil angka stok setelah "Stok Total: "
                        stock = stock_text.split(":")[-1].strip()  
                    except:
                        stock = None  # Jika gagal, set stok menjadi None

                    # Mengambil kategori produk dari elemen
                    try:
                        # Mencari elemen etalase berdasarkan class
                        etalase_elem = container.find_element(By.CSS_SELECTOR, "li.css-1i6xy22 a b")
                        etalase = etalase_elem.text.strip()  # Mengambil teks dari elemen <b>
                    except:
                        etalase = None  # Jika gagal, set etalase menjadi None

                    # Menambahkan data produk ke dalam list product_data
                    product_data.append({
                        'name_product': name,
                        'stock': stock,
                        'kategori': etalase
                    })


            # Mengonversi list product_data ke dalam DataFrame
            exsport_stock_tokped_df = pd.DataFrame(product_data)

            # Menyimpan DataFrame ke dalam file CSV
            exsport_stock_tokped_df.to_csv(self.output().path, index=False)

        except Exception as e:
            print(f"Terjadi kesalahan: {e}") # Menampilkan pesan kesalahan jika terjadi kesalahan
        
        finally:
            driver.quit() # Menutup browser

In [6]:
luigi.build([ExtractTokpedStockExsportData()], local_scheduler=True) # Menjalankan task ExtractTokpedStockExsportData

DEBUG: Checking if ExtractTokpedStockExsportData() is complete
DEBUG: Checking if ExtractTokpedExsportData() is complete
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 13744] Worker Worker(salt=9433573625, workers=1, host=zueible, username=LENOVO, pid=13744) running   ExtractTokpedStockExsportData()
INFO: [pid 13744] Worker Worker(salt=9433573625, workers=1, host=zueible, username=LENOVO, pid=13744) done      ExtractTokpedStockExsportData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Wo

True

In [6]:
class ValidateData(luigi.Task):
    def requires(self):
        return [ExtractTokpedExsportData(), ExtractTokpedStockExsportData()] # Task yang diperlukan
    
    def output(self):
        return luigi.LocalTarget('validate-raw-data/validate_data.txt')

    def run(self):
        # Buat direktori jika belum ada
        os.makedirs(os.path.dirname(self.output().path), exist_ok=True)
        
        raw_exsport_data = pd.read_csv(self.input()[0].path) # Membaca file CSV yang diekstrak sebelumnya
        raw_exsport_stock_data = pd.read_csv(self.input()[1].path) # Membaca file CSV yang diekstrak sebelumnya

        list_df = {
            'raw_exsport_data': raw_exsport_data,
            'raw_exsport_stock_data': raw_exsport_stock_data
        }
        
        with open(self.output().path, 'w', newline='') as f:
            # Check Data Shape
            f.write("==================== Check Data Shape ====================\n\n")
            result_df = []
            for name, df in list_df.items():
                n_columns = df.shape[1]
                n_rows = df.shape[0]
                result_df.append([name, n_columns, n_rows])
                
            headers_shape = ['Dataframe', 'Columns', 'Rows']
            f.write(tabulate(result_df, headers_shape, tablefmt="grid"))
            f.write("\n\n")

            # Check Data Values
            for name, df in list_df.items():
                result_val = []
                for col in df.columns:
                    col_type = df[col].dtype
                    sum_na = round(df[col].isna().sum() * 100 / len(df))
                    sum_dup = round(df.duplicated(keep=False).sum())
                    result_val.append([col, col_type, sum_na, sum_dup])

                headers_val = ['Column Name', 'Data Type', 'Missing Values (%)', 'Duplicate Values (count)']
                f.write(f"Checking Data Values: {name}\n")
                f.write(tabulate(result_val, headers_val, tablefmt="grid"))
                f.write("\n\n")

In [7]:
luigi.build([ValidateData()], local_scheduler=True) # Menjalankan task ValidateData

DEBUG: Checking if ValidateData() is complete
DEBUG: Checking if ExtractTokpedExsportData() is complete
DEBUG: Checking if ExtractTokpedStockExsportData() is complete
INFO: Informed scheduler that task   ValidateData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   DONE
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 23944] Worker Worker(salt=2009130413, workers=1, host=zueible, username=LENOVO, pid=23944) running   ValidateData()
INFO: [pid 23944] Worker Worker(salt=2009130413, workers=1, host=zueible, username=LENOVO, pid=23944) done      ValidateData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ValidateData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
D

True

In [49]:
df = pd.read_csv('extract-raw-data/exsport_tokped_raw.csv', index_col=False)
df.head()

,name_product,product_link,price_sale,price_original,discount,sold,rating,image_link
0,Exsport All Set Multipurpose Pouch - Dark Green,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
1,Exsport All Set Multipurpose Pouch - Light Brown,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
2,Exsport All Set Multipurpose Pouch - Dark Purple,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
3,Exsport All Set Multipurpose Pouch - Black,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...
4,Exsport Basic Half Moon Mini Sling Bag - Blue...,https://www.tokopedia.com/exsportstore/exsport...,Rp170.050,Rp179.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name_product    802 non-null    object 
 1   product_link    802 non-null    object 
 2   price_sale      802 non-null    object 
 3   price_original  327 non-null    object 
 4   discount        327 non-null    object 
 5   sold            720 non-null    object 
 6   rating          662 non-null    float64
 7   image_link      802 non-null    object 
dtypes: float64(1), object(7)
memory usage: 50.3+ KB


In [17]:
def extract_color(name_product):
    color_list = [
        'DARK GREEN', 'SKY BLUE', 'BIRU MUDA', 'BLUE', 'ORANGE', 'LIGHT YELLOW', 'GREEN ARMY', 'FUCHSIA',
        'LIGHT BROWN', 'DARK BLUE', 'MINT GREEN', 'DARK GREY', 'UNGU MUDA', 'FUCSHIA', 'LIGHT PINK', 'CREAM',
        'GOLD', 'DARK OLIVE', 'RED', 'GREY', 'WHITE', 'DARK PURPLE', 'SALEM', 'FUSCHIA', 'HITAM', 'LIME',
        'BROWN', 'BLUEBERRY', 'SOFT BLUE', 'BLACK', 'DARK SALEM', 'YELLOW', 'MAROON', 'DARK BROWN', 'BEIGE',
        'LIGHT BLUE', 'GREEN', 'COKELAT MUDA', 'LIGHT GREEN', 'CURRY', 'PINK', 'KREM', 'BIRU TUA', 'KHAKI',
        'PURPLE', 'EMERALD GREEN', 'DARK ORANGE', 'FUCHIA', 'NAVY', 'LIGHT PURPLE', 'OLIVE', 'LIGHT GREY'
    ]

    color_translation = {
        'KREM': 'CREAM',
        'COKELAT MUDA': 'LIGHT BROWN',
        'BIRU TUA': 'DARK BLUE',
        'HITAM': 'BLACK',
        'BIRU MUDA': 'LIGHT BLUE',
        'SALEM': 'SALMON',
        'UNGU MUDA': 'LIGHT PURPLE',
        'FUCSHIA': 'FUCHSIA',
        'FUCHSIA': 'FUCHSIA',
        'FUCHIA': 'FUCHSIA',
    }

    name_product = str(name_product).strip()

    # Konversi ke huruf besar
    name_upper = name_product.upper()

    # Ambil warna setelah tanda "-"
    color_product = name_upper.split('-')[-1].strip()

    # Urutan pengecekan:
    if color_product in color_translation:  # 1. Cek di color_translation dulu
        return color_translation[color_product]
    elif color_product in color_list:  # 2. Cek di color_list
        return color_product
    else:  # 3. Jika tidak ada di keduanya
        return None

In [50]:
df['color_product'] = df['name_product'].apply(extract_color)
df['name_product'] = df['name_product'].str.split('-').str[0].str.upper()
df.head()

,name_product,product_link,price_sale,price_original,discount,sold,rating,image_link,color_product
0,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...,DARK GREEN
1,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...,LIGHT BROWN
2,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...,DARK PURPLE
3,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,Rp141.550,Rp149.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...,BLACK
4,EXSPORT BASIC HALF MOON MINI SLING BAG,https://www.tokopedia.com/exsportstore/exsport...,Rp170.050,Rp179.000,5%,NaN,NaN,https://images.tokopedia.net/img/cache/200-squ...,BLUEBERRY


In [51]:
df['price_original'] = df['price_original'].fillna(df['price_sale'])
df['price_original'] = df['price_original'].str.replace('Rp', '').str.replace('.', '').str.strip().apply(pd.to_numeric)
df['price_sale'] = df['price_sale'].str.replace('Rp', '').str.replace('.', '').str.strip().apply(pd.to_numeric)
df['discount'] = df['discount'].fillna('Tidak ada Discount')
df['sold'] = df['sold'].str.split().str[0].fillna('0')
df['rating'] = df['rating'].fillna('Tidak ada Rating')

In [18]:
def generate_product_id(index):
    letters = "ETWS"  # Menggunakan ETWS secara berurutan
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak
    
    return letters + numbers

In [19]:
def generate_color_id(index):
    letters = "ECLR"  # Menggunakan ETWS secara berurutan
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak
    
    return letters + numbers

In [20]:
def generate_category_id(index):
    letters = "ECAT" # Menggunakan ECAT secara berurutan
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak  

In [62]:
df["product_id"] = [generate_product_id(i) for i in range(len(df))]

In [68]:
df_color = {
    'color_product' : df['color_product'].unique(),
    'color_id' : [generate_color_id(i) for i in range(len(df['color_product'].unique()))]
}

In [69]:
df_color = pd.DataFrame(df_color)
df_color.head()

,color_product,color_id
0,DARK GREEN,ECLR602
1,LIGHT BROWN,ECLR976
2,DARK PURPLE,ECLR107
3,BLACK,ECLR056
4,BLUEBERRY,ECLR112


In [66]:
df.head()

,name_product,product_link,price_sale,price_original,discount,sold,rating,image_link,color_product,product_id
0,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,DARK GREEN,ETWS420
1,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,LIGHT BROWN,ETWS485
2,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,DARK PURPLE,ETWS822
3,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,BLACK,ETWS882
4,EXSPORT BASIC HALF MOON MINI SLING BAG,https://www.tokopedia.com/exsportstore/exsport...,170050,179000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,BLUEBERRY,ETWS824


In [71]:
df = df.merge(df_color[['color_product', 'color_id']], on='color_product', how='left')

In [72]:
df.head()

,name_product,product_link,price_sale,price_original,discount,sold,rating,image_link,color_product,product_id,color_id
0,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,DARK GREEN,ETWS420,ECLR602
1,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,LIGHT BROWN,ETWS485,ECLR976
2,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,DARK PURPLE,ETWS822,ECLR107
3,EXSPORT ALL SET MULTIPURPOSE POUCH,https://www.tokopedia.com/exsportstore/exsport...,141550,149000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,BLACK,ETWS882,ECLR056
4,EXSPORT BASIC HALF MOON MINI SLING BAG,https://www.tokopedia.com/exsportstore/exsport...,170050,179000,5%,0,Tidak ada Rating,https://images.tokopedia.net/img/cache/200-squ...,BLUEBERRY,ETWS824,ECLR112


In [56]:
df_stock = pd.read_csv('extract-raw-data/exsport_stock_tokped_raw.csv', index_col=False)
df_stock.head()

,name_product,stock,kategori
0,Exsport All Set Multipurpose Pouch - Dark Green,13,Semua Etalase
1,Exsport All Set Multipurpose Pouch - Light Brown,14,Semua Etalase
2,Exsport All Set Multipurpose Pouch - Dark Purple,12,Semua Etalase
3,Exsport All Set Multipurpose Pouch - Black,14,Semua Etalase
4,Exsport Basic Half Moon Mini Sling Bag - Blueb...,21,Tas Sling Bag


In [57]:
df_stock['color_product'] = df_stock['name_product'].apply(extract_color)
df_stock['name_product'] = df_stock['name_product'].str.split('-').str[0].str.upper()
df_stock['kategori'] = df_stock['kategori'].str.upper()
df_stock.head()

,name_product,stock,kategori,color_product
0,EXSPORT ALL SET MULTIPURPOSE POUCH,13,SEMUA ETALASE,DARK GREEN
1,EXSPORT ALL SET MULTIPURPOSE POUCH,14,SEMUA ETALASE,LIGHT BROWN
2,EXSPORT ALL SET MULTIPURPOSE POUCH,12,SEMUA ETALASE,DARK PURPLE
3,EXSPORT ALL SET MULTIPURPOSE POUCH,14,SEMUA ETALASE,BLACK
4,EXSPORT BASIC HALF MOON MINI SLING BAG,21,TAS SLING BAG,BLUEBERRY


In [64]:
df_stock = df_stock.merge(df[['name_product', 'product_id']], on='name_product', how='left')

In [65]:
df_stock.head()

,name_product,stock,kategori,color_product,product_id
0,EXSPORT ALL SET MULTIPURPOSE POUCH,13,SEMUA ETALASE,DARK GREEN,ETWS420
1,EXSPORT ALL SET MULTIPURPOSE POUCH,13,SEMUA ETALASE,DARK GREEN,ETWS485
2,EXSPORT ALL SET MULTIPURPOSE POUCH,13,SEMUA ETALASE,DARK GREEN,ETWS822
3,EXSPORT ALL SET MULTIPURPOSE POUCH,13,SEMUA ETALASE,DARK GREEN,ETWS882
4,EXSPORT ALL SET MULTIPURPOSE POUCH,14,SEMUA ETALASE,LIGHT BROWN,ETWS420


In [55]:
print(list(set(df_stock['kategori'])))

['Tas Sling Bag', 'Tas Tote Bag', 'Poppin Series', 'Semua Etalase', 'TAS LAPTOP', 'STARTER TENNIS SERIES', 'SWEET VALENTINE GIFTS', 'BASIC COLLECTION', 'JUNIOR SERIES THE FUTURE ME', 'Vacation Series', 'Tas Mini Backpack', 'FILL IN COLOUR SERIES', 'Tas Backpack']


In [8]:
a = pd.read_csv('extract-raw-data/exsport_tokped_raw.csv')

b = a['name_product'].apply(extract_color).dropna().unique()

In [9]:
print(b)

['DARK GREEN' 'LIGHT BROWN' 'DARK PURPLE' 'BLACK' 'BLUEBERRY'
 'LIGHT YELLOW' 'BEIGE' 'GREEN ARMY' 'DARK BLUE' 'MAROON' 'LIGHT PINK'
 'DARK GREY' 'DARK OLIVE' 'PURPLE' 'EMERALD GREEN' 'FUCHSIA' 'PINK'
 'CREAM' 'NAVY' 'RED' 'LIGHT BLUE' 'LIGHT PURPLE' 'CURRY' 'SKY BLUE'
 'GREEN' 'SALMON' 'LIME' 'DARK SALEM' 'BLUE' 'BROWN' 'MINT GREEN'
 'LIGHT GREY' 'OLIVE' 'GREY' 'DARK ORANGE' 'ORANGE' 'YELLOW' 'DARK BROWN'
 'LIGHT GREEN' 'GOLD']


In [10]:
c = a['name_product'].str.split('-').str[0].str.upper().unique()

In [11]:
c

array(['EXSPORT ALL SET MULTIPURPOSE POUCH ',
       'EXSPORT BASIC HALF MOON MINI SLING BAG ',
       'EXSPORT BASIC HALF MOON SLING BAG L ',
       'EXSPORT JOYFUL PACK BUNDLE', 'EXSPORT FESTIVE PACK BUNDLE',
       'EXSPORT SPARKS PACK BUNDLE', 'EXSPORT RETURN PACK BUNDLE',
       'EXSPORT HOMECOMING PACK BUNDLE',
       'EXSPORT CLASSIC "RR01 1979" BACKPACK ',
       'EXSPORT EVERYDAY LAPTOP COMPARTMENT BAG ',
       'EXSPORT EASY GOING MINI BACKPACK ', 'EXSPORT KUMARA TOTE BAG ',
       'EXSPORT DAILY CARRY HAND BAG ',
       'EXSPORT FLOW AND GO 102 TOTE BAG ', 'EXSPORT EVERYDAY LANYARD ',
       'EXSPORT SHOULDER PADS EXTENSION ', 'EXSPORT SQUARE COIN POUCH ',
       'EXSPORT DAILY LAPTOP HANDBAG ', 'EXSPORT FLEUR',
       'EXSPORT GO ACTIVE MINI SLING BAG ',
       'EXSPORT BASIC CUBICAL SLING POUCH ',
       'EXSPORT TRIANGLE COIN POUCH ', 'EXSPORT BIMBLE COIN POUCH ',
       'TAS EXSPORT BIMBLE COIN POUCH ',
       'TAS EXSPORT BASIC TWO WAY CARRY ',
       'TAS EXSPORT REROU

In [31]:
class TransformTokpedExportData(luigi.Task):
    def requires(self):
        return ExtractTokpedExsportData(), ExtractTokpedStockExsportData()
    
    def output(self):
        return [
            luigi.LocalTarget('transform-data/data_product_exsport.csv'),
            luigi.LocalTarget('transform-data/data_stock_exsport.csv'),
            luigi.LocalTarget('transform-data/data_name_exsport.csv'),
            luigi.LocalTarget('transform-data/data_color_exsport.csv'),
            luigi.LocalTarget('transform-data/data_category_exsport.csv')
        ]
    
    def run(self):
        # Mengekstrak data dari file csv
        df_product = pd.read_csv('/extract-raw-data/exsport_tokped_raw.csv')
        df_stock = pd.read_csv('/extract-raw-data/exsport_stock_tokped_raw.csv')

        # Transformasi data
        # Ekstrak color dari nama produk
        df_color = {
            'color_id' : [generate_color_id(i) for i in range(len(df_product['name'].apply(extract_color).dropna().unique()))],
            'color' : df_product['name'].apply(extract_color).dropna().unique()
        }
        df_color = pd.DataFrame(df_color)

        # Ekstrak kategori dari stock
        df_category = {
            'category_id' : [generate_category_id(i) for i in range(len(df_stock['category'].dropna().unique()))],
            'category' : df_stock['category'].str.upper().dropna().unique()
        }
        df_category = pd.DataFrame(df_category)

        # Ekstrak nama produk
        df_name = {
            'product_id' : [generate_product_id(i) for i in range(len(df_product['name'].dropna().unique()))],
            'name_product' : df_product['name_product'].str.split('-').str[0].str.upper().dropna().unique()
        }
        df_name = pd.DataFrame(df_name)

        # Transformasi df_product
        df_product['color_id'] = df_product['name_product'].apply(extract_color)
        df_product['name_product'] = df_product['name_product'].str.split('-').str[0].str.upper()
        df_product = df_product.merge(df_name, on='name_product', how='left')
        df_product = df_product.merge(df_color, on='color_id', how='left')
        df_product = df_product.drop(columns=['name_product', 'color_id'])
        df_product['price_original'] = df_product['price_original'].fillna(df_product['price_sale'])
        df_product['price_original'] = df_product.str.replace('Rp', '').str.replace('.', '').str.strip().apply(pd.to_numeric)
        df_product['price_sale'] = df_product['price_sale'].str.replace('Rp', '').str.replace('.', '').str.strip().apply(pd.to_numeric)
        df_product['discount'] = df_product['discount'].fillna('No Discount')
        df_product['sold'] = df_product['sold'].fillna('0')
        df_product['sold'] = df_product['sold'].str.replace('Terjual', '').str.strip().astype(str)
        df_product['rating'] = df_product['rating'].fillna('No Discount')
        df_product = df_product[['product_id', 'color_id', 'price_original', 'price_sale', 'discount', 'sold', 'rating']]

        # Transformasi df_stock
        df_stock['color'] = df_stock['name_product'].apply(extract_color)
        df_stock['name_product'] = df_stock['name_product'].str.split('-').str[0].str.upper()
        df_stock = df_stock.merge(df_name, on='name_product', how='left')
        df_stock = df_stock.merge(df_category, on='category', how='left')
        df_stock = df_stock.merge(df_color, on='color', how='left')
        df_stock['stock'] = df_stock['stock'].fillna('0')
        df_stock['stock'] = df_stock['stock'].str.replace('Sisa', '').str.strip().astype(int)
        df_stock = df_stock[['product_id', 'category_id', 'color_id', 'stock']]

        # Menyimpan data
        os.makedirs('transform-data', exist_ok=True)
        df_product.to_csv('transform-data/data_product_exsport.csv', index=False)
        df_stock.to_csv('transform-data/data_stock_exsport.csv', index=False)
        df_name.to_csv('transform-data/data_name_exsport.csv', index=False)
        df_color.to_csv('transform-data/data_color_exsport.csv', index=False)
        df_category.to_csv('transform-data/data_category_exsport.csv', index=False)

In [32]:
luigi.build([TransformTokpedExportData()], local_scheduler=True)

DEBUG: Checking if TransformTokpedExportData() is complete
DEBUG: Checking if ExtractTokpedExsportData() is complete
DEBUG: Checking if ExtractTokpedStockExsportData() is complete
INFO: Informed scheduler that task   TransformTokpedExportData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractTokpedStockExsportData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractTokpedExsportData__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 3
INFO: [pid 14092] Worker Worker(salt=017122577, workers=1, host=zueible, username=LENOVO, pid=14092) running   ExtractTokpedExsportData()
INFO: Worker Worker(salt=017122577, workers=1, host=zueible, username=LENOVO, pid=14092) was stopped. Shutting down Keep-Alive thread


KeyboardInterrupt: 

In [21]:
import pandas as pd
import random

def extract_category(name_product):
    name_lower = name_product.lower()
    if any (keyword in name_lower for keyword in ['laptop', 'sleeve']):
        return 'TAS LAPTOP'
    elif any (keyword in name_lower for keyword in ['sling bag', 'sling pouch']):
        return 'TAS SLING BAG'
    elif any (keyword in name_lower for keyword in ['backpack', 'rucksack']):
        return 'TAS BACKPACK'
    elif any (keyword in name_lower for keyword in ['pouch']):
        return 'TAS POUCH'
    elif any (keyword in name_lower for keyword in ['tote bag']):
        return 'TAS TOTE BAG'
    else:
        return 'SEMUA ETALASE'

def extract_color(name_product):
    color_list = [
        'DARK GREEN', 'SKY BLUE', 'BIRU MUDA', 'BLUE', 'ORANGE', 'LIGHT YELLOW', 'GREEN ARMY', 'FUCHSIA',
        'LIGHT BROWN', 'DARK BLUE', 'MINT GREEN', 'DARK GREY', 'UNGU MUDA', 'FUCSHIA', 'LIGHT PINK', 'CREAM',
        'GOLD', 'DARK OLIVE', 'RED', 'GREY', 'WHITE', 'DARK PURPLE', 'SALEM', 'FUSCHIA', 'HITAM', 'LIME',
        'BROWN', 'BLUEBERRY', 'SOFT BLUE', 'BLACK', 'DARK SALEM', 'YELLOW', 'MAROON', 'DARK BROWN', 'BEIGE',
        'LIGHT BLUE', 'GREEN', 'COKELAT MUDA', 'LIGHT GREEN', 'CURRY', 'PINK', 'KREM', 'BIRU TUA', 'KHAKI',
        'PURPLE', 'EMERALD GREEN', 'DARK ORANGE', 'FUCHIA', 'NAVY', 'LIGHT PURPLE', 'OLIVE', 'LIGHT GREY'
    ]

    color_translation = {
        'KREM': 'CREAM',
        'COKELAT MUDA': 'LIGHT BROWN',
        'BIRU TUA': 'DARK BLUE',
        'HITAM': 'BLACK',
        'BIRU MUDA': 'LIGHT BLUE',
        'SALEM': 'SALMON',
        'UNGU MUDA': 'LIGHT PURPLE',
        'FUCSHIA': 'FUCHSIA',
        'FUCHSIA': 'FUCHSIA',
        'FUCHIA': 'FUCHSIA',
    }

    name_product = str(name_product).strip()

    # Konversi ke huruf besar
    name_upper = name_product.upper()

    # Ambil warna setelah tanda "-"
    color_product = name_upper.split('-')[-1].strip()

    # Urutan pengecekan:
    if color_product in color_translation:  # 1. Cek di color_translation dulu
        return color_translation[color_product]
    elif color_product in color_list:  # 2. Cek di color_list
        return color_product
    else:  # 3. Jika tidak ada di keduanya
        return 'Tidak ada spesifikasi warna'

def generate_product_id(index):
    letters = "ETWS"  # Menggunakan ETWS secara berurutan
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak
    product_id = letters + numbers
    
    return product_id

def generate_color_id(index):
    letters = "ECLR"
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak  
    category_id = letters + numbers  
    return category_id

def generate_category_id(index):
    letters = "ECAT"
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak  
    category_id = letters + numbers  
    return category_id

In [22]:
def generate_category_id(index):
    letters = "ECAT"
    numbers = ''.join(random.choices("0123456789", k=3))  # 3 angka acak  
    category_id = letters + numbers  
    return category_id


In [23]:
df_stock = pd.read_csv('extract-raw-data/exsport_stock_tokped.csv')
df_product = pd.read_csv('extract-raw-data/exsport_tokped_raw.csv')

In [24]:
df_color = {
    'color_id' : [generate_color_id(i) for i in range(len(df_product['name_product'].apply(extract_color).dropna().unique()))],
    'color' : df_product['name_product'].apply(extract_color).dropna().unique()
}
df_color = pd.DataFrame(df_color)

# Ekstrak kategori dari stock
df_category = {
    'category_id' : [generate_category_id(i) for i in range(len(df_stock['category'].unique()))],
    'category' : df_stock['category'].str.upper().dropna().unique()
}
df_category = pd.DataFrame(df_category)

# Ekstrak nama produk
name_product_unique = df_product['name_product'].str.replace('"', '').str.split('-').str[0].str.upper().dropna().unique()

df_name = pd.DataFrame({
    'product_id': [generate_product_id(i) for i in range(len(name_product_unique))],
    'name_product': list(name_product_unique)  # Pastikan ini berbentuk list agar panjangnya sama
})

df_name = pd.DataFrame(df_name)

In [25]:
df_color.to_csv('data_color_exsport.csv', index=False)

In [19]:
df_category.to_csv('data_category_exsport.csv', index=False)

In [20]:
df_name.to_csv('data_name_exsport.csv', index=False)

In [7]:
import random
import pandas as pd
from faker import Faker
from datetime import datetime
from datetime import date

# Inisialisasi Faker untuk Indonesia
fake = Faker("id_ID")

# Load data produk
df_products = pd.read_csv("transform-data/data_product_exsport.csv")

# Konversi kolom sold ke integer (jika sebelumnya dalam format "30+" atau "50+")
df_products['sold'] = df_products['sold'].astype(int)

# Hanya ambil produk yang memiliki penjualan (> 0)
df_products = df_products[df_products['sold'] > 0]

# Load data customer
df_customers = pd.read_csv("customer_profiles.csv")
customer_ids = df_customers['customer_id'].tolist()

# Daftar metode pengiriman
shipping_methods = ["JNE", "J&T", "SiCepat"]

# Daftar metode pembayaran
payment_methods = ["Transfer Bank - BCA", "Transfer Bank - Mandiri", "Transfer Bank - BNI", "Transfer Bank - BRI", "GoPay", "OVO", "DANA", "COD"]

# Fungsi untuk membuat Order ID unik
def generate_order_id(index):
    letters = "ORDT"
    numbers = ''.join(random.choices("0123456789", k=5))  # 5 angka acak  
    return letters + numbers  

# Fungsi untuk menghasilkan tanggal pesanan
def generate_order_date():
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2025, 2, 28)
    return fake.date_between(start_date=start_date, end_date=end_date)

# Fungsi untuk menentukan status berdasarkan tanggal
def determine_order_status(order_date):
    if order_date < date(2025, 2, 15):  # Ubah datetime ke date
        return "Selesai"
    elif order_date < date(2025, 2, 25):
        return "Dikirim"
    else:
        return "Pending"

# Fungsi untuk membuat order
def create_order(customer_id, sold_tracker):
    order_id = generate_order_id(random.randint(10000, 99999))
    order_date = generate_order_date()
    order_status = determine_order_status(order_date)
    shipping_method = random.choice(shipping_methods)
    payment_method = random.choice(payment_methods)

    num_products = random.randint(1, 3)
    selected_products = []
    total_price = 0

    # Ambil daftar produk yang masih tersedia
    available_products = [p for p, s in sold_tracker.items() if s > 0]
    random.shuffle(available_products)

    for _ in range(num_products):
        if not available_products:
            break
        product_id = available_products.pop()
        quantity = random.randint(1, min(3, sold_tracker[product_id]))
        price_sale = df_products.loc[df_products['product_id'] == product_id, 'price_sale'].values[0]

        selected_products.append(f"{product_id} (x{quantity}, Rp{price_sale})")
        total_price += price_sale * quantity
        sold_tracker[product_id] -= quantity

        # Hapus produk dari daftar jika stoknya habis
        if sold_tracker[product_id] == 0 and product_id in available_products:
            available_products.remove(product_id)

    if not selected_products:  # Jika tidak ada produk yang bisa dibeli, skip order
        return None

    return {
        "Order_ID": order_id,
        "Customer_ID": customer_id,
        "Products": "; ".join(selected_products),
        "Order_Date": order_date.strftime("%Y-%m-%d"),
        "Order_Status": order_status,
        "Shipping_Method": shipping_method,
        "Payment_Method": payment_method,
        "Total_Price": total_price
    }

# Fungsi untuk membuat data order
def generate_order_data(num_orders=3000):
    data = []
    sold_tracker = df_products.set_index('product_id')['sold'].to_dict()

    # Buat order untuk 1000 customer pertama (pastikan mereka melakukan setidaknya 1 order)
    selected_customers = random.sample(customer_ids, 1000)
    for customer_id in selected_customers:
        order_data = create_order(customer_id, sold_tracker)
        if order_data:
            data.append(order_data)

    # Buat sisa order secara random (hingga mencapai 3000)
    for _ in range(num_orders - len(data)):
        customer_id = random.choice(customer_ids)
        order_data = create_order(customer_id, sold_tracker)
        if order_data:
            data.append(order_data)

    return pd.DataFrame(data)

# Generate 3000 data order
df_orders = generate_order_data(3000)

# Simpan ke CSV
df_orders.to_csv("order_data.csv", index=False)

In [2]:
import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta

# Inisialisasi Faker untuk Indonesia
fake = Faker("id_ID")

# Load data produk
df_products = pd.read_csv("transform-data/data_product_exsport.csv")

# Filter hanya produk yang memiliki penjualan (> 0)
df_products = df_products[df_products['sold']> 0]

# Load data customer
df_customers = pd.read_csv("customer_profiles.csv")
customer_ids = df_customers['customer_id'].tolist()

# Daftar metode pengiriman
shipping_methods = ["JNE", "J&T", "SiCepat"]

# Daftar metode pembayaran
payment_methods = [
    "Transfer Bank - BCA", "Transfer Bank - Mandiri", "Transfer Bank - BNI", "Transfer Bank - BRI", 
    "GoPay", "OVO", "DANA", "COD"
]

# Fungsi untuk membuat Order ID unik
def generate_order_id():
    return "ORD" + ''.join(random.choices("0123456789", k=6))

# Fungsi untuk menghasilkan tanggal pesanan
def generate_order_date():
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2025, 2, 28)
    return fake.date_between(start_date=start_date, end_date=end_date)

# Fungsi untuk menentukan status berdasarkan tanggal
def determine_order_status(order_date):
    if order_date < datetime(2025, 2, 15).date():
        return "Selesai"
    elif order_date < datetime(2025, 2, 25).date():
        return "Dikirim"
    else:
        return "Pending"

# Fungsi untuk membuat data order
def create_order(customer_id, sold_tracker):
    order_id = generate_order_id()
    order_date = generate_order_date()
    order_status = determine_order_status(order_date)
    shipping_method = random.choice(shipping_methods)
    payment_method = random.choice(payment_methods)
    
    num_products = random.randint(1, 3)  # Maks 3 produk dalam satu order
    available_products = [p for p, s in sold_tracker.items() if s > 0]
    random.shuffle(available_products)
    
    order_data = []
    for _ in range(num_products):
        if not available_products:
            break
        
        product_id = available_products.pop()
        quantity = random.randint(1, min(3, sold_tracker[product_id]))
        price_sale = df_products.loc[df_products['product_id'] == product_id, 'price_sale'].values[0]
        
        order_data.append({
            "Order_ID": order_id,
            "Customer_ID": customer_id,
            "Product_ID": product_id,
            "Quantity": quantity,
            "Order_Date": order_date.strftime("%Y-%m-%d"),
            "Order_Status": order_status,
            "Shipping_Method": shipping_method,
            "Payment_Method": payment_method,
            "Total_Price": price_sale * quantity
        })
        
        sold_tracker[product_id] -= quantity
    
    return order_data

# Fungsi utama untuk membuat dataset order
def generate_order_data(num_orders=3000):
    data = []
    sold_tracker = df_products.set_index('product_id')['sold'].astype(str).str.replace('+', '', regex=False).astype(int).to_dict()
    
    selected_customers = random.sample(customer_ids, 1000)  # 1000 customer harus order minimal 1 kali
    
    for customer_id in selected_customers:
        order_data = create_order(customer_id, sold_tracker)
        if order_data:
            data.extend(order_data)
    
    remaining_orders = num_orders - len(data)
    for _ in range(remaining_orders):
        customer_id = random.choice(customer_ids)
        order_data = create_order(customer_id, sold_tracker)
        if order_data:
            data.extend(order_data)
    
    return pd.DataFrame(data)

# Generate 3000 data order
df_orders = generate_order_data(3000)

# Simpan ke CSV
df_orders.to_csv("order_data.csv", index=False)


In [5]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime

# Inisialisasi Faker untuk bahasa Indonesia
fake = Faker("id_ID")

# Load data order dan data produk
df_orders = pd.read_csv("order_data.csv")
df_products = pd.read_csv("transform-data/data_product_exsport.csv")

# Hanya produk yang memiliki rating (tidak termasuk "No Rating")
df_products = df_products[df_products['rating'] != "No Rating"].copy()
df_products['rating'] = df_products['rating'].astype(float)

# Mapping product_id ke rating
product_rating_map = df_products.set_index("product_id")["rating"].to_dict()

# Contoh review dalam bahasa Indonesia berdasarkan kategori produk (tas)
review_texts = [
    "Tasnya sangat bagus dan kuat!",
    "Bahan berkualitas, jahitan rapi. Sangat puas!",
    "Ukuran sesuai deskripsi, bisa muat banyak barang.",
    "Warnanya cantik, persis seperti di foto.",
    "Pengiriman cepat dan kemasannya rapi!",
    "Lumayan, tapi kualitas tali kurang kuat.",
    "Tasnya nyaman dipakai, cocok untuk sehari-hari.",
    "Harga sesuai dengan kualitas, tidak mengecewakan.",
    "Bagus, tapi harap diperhatikan kualitas resletingnya.",
    "Modelnya trendi, saya suka!"
]

# Fungsi untuk menghasilkan review
def generate_review(order):
    order_id = order["Order_ID"]
    customer_id = order["Customer_ID"]
    product_id = order["Product_ID"]
    order_date = datetime.strptime(order["Order_Date"], "%Y-%m-%d").date()  # Konversi ke date
    
    max_rating = product_rating_map.get(product_id, 5)  # Pastikan menggunakan variabel yang benar
    rating = round(random.uniform(1, max_rating) * 2) / 2  # Rating dalam kelipatan 0.5
    
    return {
        "Order_ID": order_id,
        "Reviewer_ID": customer_id,
        "Product_ID": product_id,
        "Review_Text": random.choice(review_texts),
        "Star_Rating": rating,
        "Review_Date": fake.date_between(start_date=order_date)  # Tanggal review setelah order
    }

# Generate review data
df_reviews = df_orders.apply(generate_review, axis=1)

df_reviews = pd.DataFrame(df_reviews.tolist())

# Simpan ke CSV
df_reviews.to_csv("review_data.csv", index=False)


In [ ]:

# class ExtractTokpedStockExsportData(luigi.Task):
#     def requires(self):
#         return ExtractTokpedExsportData() # Task yang diperlukan
    
#     def output(self):
#         BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "extract-raw-data"))
#         OUTPUT_PATH = os.path.join(BASE_DIR, "exsport_stock_tokped.csv")

#         return luigi.LocalTarget(OUTPUT_PATH) # Tempat penyimpanan data yang diekstrak


#     def run(self):
#         extract_data = pd.read_csv(self.input().path) # Membaca data produk yang diekstrak sebelumnya
#         pages = extract_data['product_link'].tolist()

#         options = webdriver.ChromeOptions()
#         options.add_argument('--disable-blink-features=AutomationControlled')
#         options.add_experimental_option('useAutomationExtension', False)
#         options.add_experimental_option("excludeSwitches", ["enable-automation"])
#         options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
#         # options.add_argument("--headless=new")  # Headless mode untuk mencegah deteksi bot
#         options.page_load_strategy = 'eager'  # Mempercepat loading

#         driver = webdriver.Chrome(options=options)

#         stock_data = []

#         try:
#             for page in pages:  # Loop untuk setiap link dalam list
#                 driver.get(page)
#                 time.sleep(5)   

#                 try:
#                     WebDriverWait(driver, 30).until(
#                         EC.presence_of_element_located((By.TAG_NAME, 'body'))
#                     )
#                 except:
#                     print(f"Gagal memuat halaman {page}. Lewati")
#                     continue  # Lanjutkan ke halaman berikutnya jika halaman tidak dapat dimuat

#                         # Scroll untuk memuat halaman
#                 for _ in range(5):
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     time.sleep(2)
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
#                     time.sleep(2)  # Tunggu sebentar untuk memastikan elemen muncul

#                 try:
#                     container = driver.find_element(By.CLASS_NAME, "css-856ghu")
#                 except:
#                     print(f"Elemen utama tidak ditemukan di {page}")
#                     continue  # Skip ke URL berikutnya jika elemen tidak ada

#                 try:
#                     name = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDetailProductName']").text.strip()
#                 except:
#                     name = None

#                 try:
#                     stock_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='stock-label']")
#                     stock_text = stock_elem.text.strip()
#                     stock = stock_text.split(":")[-1].strip() if "Stok Total" in stock_text else stock_text
#                 except:
#                     stock = None

#                 try:
#                     etalase_elem = container.find_element(By.CSS_SELECTOR, "li[class*='css-'] a b")
#                     etalase = etalase_elem.text.strip()
#                 except:
#                     etalase = None

#                 # try:
#                 #     description_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDescriptionProduk']")
#                 #     text_description = description_elem.text.strip()
#                 # except:
#                 #     text_description = None

#                 stock_data.append({
#                     'name_product': name,
#                     'stock': stock,
#                     'kategori': etalase
#                 })

#             exsport_stock_tokped_df = pd.DataFrame(stock_data)
#             exsport_stock_tokped_df.to_csv(self.output().path, index=False)
            
#         except Exception as e:
#             print(f"Terjadi kesalahan: {e}")

#         finally:
#             driver.quit() # Menutup browser

# class ExtractTokpedStockExsportData(luigi.Task):
#     def requires(self):
#         return ExtractTokpedExsportData()

#     def output(self):
#         BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "extract-raw-data"))
#         OUTPUT_PATH = os.path.join(BASE_DIR, "exsport_stock_tokped.csv")
#         return luigi.LocalTarget(OUTPUT_PATH)

#     def run(self):
#         extract_data = pd.read_csv(self.input().path)
#         pages = extract_data['product_link'].tolist()

#         options = webdriver.ChromeOptions()
#         options.add_argument('--disable-blink-features=AutomationControlled')
#         options.add_experimental_option('useAutomationExtension', False)
#         options.add_experimental_option("excludeSwitches", ["enable-automation"])
#         options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
#         options.page_load_strategy = 'eager'

#         driver = webdriver.Chrome(options=options)
#         driver.set_page_load_timeout(60)
#         driver.implicitly_wait(10)

#         stock_data = []

#         def slow_scroll(driver):
#             for _ in range(5):
#                 driver.execute_script("window.scrollBy(0, document.body.scrollHeight/5);")
#                 time.sleep(1.5)

#         try:
#             for idx, page in enumerate(pages):
#                 # Restart driver setiap 10 halaman untuk menghindari crash
#                 if idx > 0 and idx % 10 == 0:
#                     driver.quit()
#                     driver = webdriver.Chrome(options=options)

#                 try:
#                     driver.get(page)
#                     time.sleep(random.uniform(5, 7))  # Random delay untuk menghindari deteksi bot
                    
#                     # Tunggu hingga elemen utama produk muncul
#                     WebDriverWait(driver, 30).until(
#                         EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='lblPDPDetailProductName']"))
#                     )
#                     print(f"✅ Halaman berhasil dimuat: {page}")

#                     # Scroll untuk memuat konten lebih baik
#                     slow_scroll(driver)

#                     # Ambil elemen utama
#                     try:
#                         container = driver.find_element(By.CLASS_NAME, "css-856ghu")
#                     except:
#                         print(f"⚠️ Elemen utama tidak ditemukan di {page}")
#                         continue

#                     # Ambil Nama Produk
#                     try:
#                         name = container.find_element(By.CSS_SELECTOR, "[data-testid='lblPDPDetailProductName']").text.strip()
#                     except:
#                         name = None

#                     # Ambil Stok Produk
#                     try:
#                         stock_elem = container.find_element(By.CSS_SELECTOR, "[data-testid='stock-label']")
#                         stock_text = stock_elem.text.strip()
#                         stock = stock_text.split(":")[-1].strip() if "Stok Total" in stock_text else stock_text
#                     except:
#                         stock = None

#                     # Ambil Kategori Produk
#                     try:
#                         etalase_elem = container.find_element(By.CSS_SELECTOR, "li[class*='css-'] a b")
#                         etalase = etalase_elem.text.strip()
#                     except:
#                         etalase = None

#                     stock_data.append({
#                         'name_product': name,
#                         'stock': stock,
#                         'kategori': etalase
#                     })

#                 except Exception as e:
#                     print(f"❌ Gagal mengambil data dari {page} - {str(e)}")

#             # Simpan hasil scraping ke CSV
#             exsport_stock_tokped_df = pd.DataFrame(stock_data)
#             exsport_stock_tokped_df.to_csv(self.output().path, index=False)

#         finally:
#             driver.quit()


In [2]:
import pandas as pd

In [3]:
dd = pd.read_csv('clean-data/data_user_tokped.csv')

df = dd['customer_id'].tolist()
print(df)

['CSTM28388', 'CSTM08617', 'CSTM46166', 'CSTM27548', 'CSTM63120', 'CSTM49502', 'CSTM97454', 'CSTM75102', 'CSTM59012', 'CSTM26638', 'CSTM09655', 'CSTM13746', 'CSTM61025', 'CSTM99567', 'CSTM74997', 'CSTM06092', 'CSTM73133', 'CSTM01320', 'CSTM24442', 'CSTM09461', 'CSTM77524', 'CSTM75994', 'CSTM31824', 'CSTM51871', 'CSTM47646', 'CSTM40507', 'CSTM55568', 'CSTM90211', 'CSTM86871', 'CSTM40173', 'CSTM40072', 'CSTM26777', 'CSTM51770', 'CSTM40841', 'CSTM02356', 'CSTM35394', 'CSTM16728', 'CSTM46652', 'CSTM98779', 'CSTM51360', 'CSTM10127', 'CSTM26995', 'CSTM77640', 'CSTM31055', 'CSTM22472', 'CSTM27084', 'CSTM18226', 'CSTM93437', 'CSTM67465', 'CSTM41906', 'CSTM29796', 'CSTM29809', 'CSTM99422', 'CSTM17629', 'CSTM81688', 'CSTM69207', 'CSTM26349', 'CSTM80514', 'CSTM09944', 'CSTM11143', 'CSTM53034', 'CSTM88383', 'CSTM44935', 'CSTM54739', 'CSTM57956', 'CSTM74508', 'CSTM89196', 'CSTM02954', 'CSTM05004', 'CSTM02083', 'CSTM56047', 'CSTM17478', 'CSTM05700', 'CSTM49919', 'CSTM78322', 'CSTM51778', 'CSTM65208'

In [7]:
import psycopg2
import pandas as pd

# Koneksi awal untuk membuat database
def create_database():
    try:
        conn = psycopg2.connect(dbname='postgres', user='postgres', password='qwerty123', host='localhost', port='5432')
        conn.autocommit = True
        cursor = conn.cursor()

        cursor.execute("SELECT 1 FROM pg_database WHERE datname = 'cobaexsport_db';")
        exists = cursor.fetchone()
        if not exists:
            cursor.execute("CREATE DATABASE cobaexsport_db;")
            print("Database 'cobaexsport_db' created successfully!")
        else:
            print("Database 'cobaexsport_db' already exists.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error creating database: {e}")

# Koneksi ke database baru dan buat tabel
def create_tables():
    try:
        conn = psycopg2.connect(dbname='cobaexsport_db', user='postgres', password='qwerty123', host='localhost', port='5432')
        cursor = conn.cursor()

        # Buat tabel-tabel dengan relasi
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS data_category_exsport (
            category_id SERIAL PRIMARY KEY,
            category VARCHAR(255) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS data_color_exsport (
            color_id SERIAL PRIMARY KEY,
            color VARCHAR(255) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS data_name_exsport (
            product_id SERIAL PRIMARY KEY,
            name_product VARCHAR(255) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS data_product_exsport (
            product_id SERIAL PRIMARY KEY,
            color_id INT REFERENCES data_color_exsport(color_id),
            price_original FLOAT,
            price_sale FLOAT,
            discount FLOAT,
            sold INT,
            rating FLOAT
        );

        CREATE TABLE IF NOT EXISTS data_stock_exsport (
            product_id INT REFERENCES data_product_exsport(product_id),
            category_id INT REFERENCES data_category_exsport(category_id),
            color_id INT REFERENCES data_color_exsport(color_id),
            stock INT,
            PRIMARY KEY (product_id, category_id, color_id)
        );

        CREATE TABLE IF NOT EXISTS data_user_tokped (
            customer_id SERIAL PRIMARY KEY,
            name VARCHAR(255),
            age INT,
            gender VARCHAR(10),
            location VARCHAR(255),
            browsing_history TEXT,
            purchase_history TEXT
        );

        CREATE TABLE IF NOT EXISTS data_order_tokped (
            order_id SERIAL PRIMARY KEY,
            customer_id INT REFERENCES data_user_tokped(customer_id),
            product_id INT REFERENCES data_product_exsport(product_id),
            quantity INT,
            order_date DATE,
            order_status VARCHAR(50),
            shipping_method VARCHAR(50),
            payment_method VARCHAR(50),
            total_price FLOAT
        );

        CREATE TABLE IF NOT EXISTS data_review_order_tokped (
            order_id INT REFERENCES data_order_tokped(order_id),
            reviewer_id INT REFERENCES data_user_tokped(customer_id),
            product_id INT REFERENCES data_product_exsport(product_id),
            review_text TEXT,
            rating FLOAT,
            review_date DATE,
            PRIMARY KEY (order_id, reviewer_id, product_id)
        );
        """)

        conn.commit()
        cursor.close()
        conn.close()
        print("Tables created successfully!")
    
    except Exception as e:
        print(f"Error creating tables: {e}")

# Fungsi untuk memasukkan data dari CSV ke tabel
def load_data_from_csv(table_name, file_path):
    try:
        conn = psycopg2.connect(dbname='cobaexsport_db', user='postgres', password='qwerty123', host='localhost', port='5432')
        cursor = conn.cursor()

        # Baca CSV
        df = pd.read_csv(file_path)
        
        # Generate query untuk insert data
        columns = ", ".join(df.columns)
        values_placeholder = ", ".join(["%s"] * len(df.columns))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholder}) ON CONFLICT DO NOTHING;"

        for _, row in df.iterrows():
            cursor.execute(query, tuple(row))

        conn.commit()
        cursor.close()
        conn.close()
        print(f"Data inserted into {table_name} successfully!")

    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

if __name__ == "__main__":
    create_database()
    create_tables()
    
    # Masukkan data dari CSV ke database
    load_data_from_csv("data_category_exsport", "clean-data/data_category_exsport.csv")
    load_data_from_csv("data_color_exsport", "clean-data/data_color_exsport.csv")
    load_data_from_csv("data_name_exsport", "clean-data/data_name_exsport.csv")
    load_data_from_csv("data_product_exsport", "clean-data/data_product_exsport.csv")
    load_data_from_csv("data_stock_exsport", "clean-data/data_stock_exsport.csv")
    load_data_from_csv("data_user_tokped", "clean-data/data_user_tokped.csv")
    load_data_from_csv("data_order_tokped", "clean-data/data_order_tokped.csv")
    load_data_from_csv("data_review_order_tokped", "clean-data/data_review_order_tokped.csv")

    print("All processes completed successfully!")


Database 'cobaexsport_db' created successfully!
Tables created successfully!
Error inserting data into data_category_exsport: invalid input syntax for type integer: "ECAT190"
LINE 1: ..._category_exsport (category_id, category) VALUES ('ECAT190',...
                                                             ^

Error inserting data into data_color_exsport: invalid input syntax for type integer: "ECLR919"
LINE 1: ...INTO data_color_exsport (color_id, color) VALUES ('ECLR919',...
                                                             ^

Error inserting data into data_name_exsport: invalid input syntax for type integer: "ETWS885"
LINE 1: ...a_name_exsport (product_id, name_product) VALUES ('ETWS885',...
                                                             ^

Error inserting data into data_product_exsport: invalid input syntax for type integer: "ETWS885"
LINE 1: ...inal, price_sale, discount, sold, rating) VALUES ('ETWS885',...
                                               

In [21]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Fungsi untuk membuat koneksi ke PostgreSQL
def create_connection():
    try:
        conn = psycopg2.connect(dbname='cobaexsport_db', user='postgres', password='qwerty123', host='localhost', port='5432')
        print("Connection to PostgreSQL DB successful")
        return conn
    except Exception as e:
        print(f"Error: {e}")
        return None

# Fungsi untuk membuat tabel di database PostgreSQL
def create_tables():
    try:
        conn = create_connection()
        cursor = conn.cursor()

        # Membuat tabel di PostgreSQL
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS data_category_exsport (
            category_id VARCHAR(20) PRIMARY KEY,
            category VARCHAR(255) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS data_color_exsport (
            color_id VARCHAR(20) PRIMARY KEY,
            color VARCHAR(255) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS data_name_exsport (
            product_id VARCHAR(20) PRIMARY KEY,
            name_product VARCHAR(255) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS data_product_exsport (
            product_id VARCHAR(20) PRIMARY KEY,
            color_id VARCHAR(20) REFERENCES data_color_exsport(color_id),
            price_original INT,
            price_sale INT,
            discount VARCHAR(255),  -- Tetap menggunakan string untuk discount
            sold INT,
            rating VARCHAR(255)  -- Tetap menggunakan string untuk rating
        );

        CREATE TABLE IF NOT EXISTS data_stock_exsport (
            product_id VARCHAR(20) REFERENCES data_product_exsport(product_id),
            category_id VARCHAR(20) REFERENCES data_category_exsport(category_id),
            color_id VARCHAR(20) REFERENCES data_color_exsport(color_id),
            stock INT,
            PRIMARY KEY (product_id, category_id, color_id)
        );

        CREATE TABLE IF NOT EXISTS data_user_tokped (
            customer_id VARCHAR(20) PRIMARY KEY,
            name VARCHAR(255),
            age INT,
            gender VARCHAR(10),
            location VARCHAR(255),
            browsing_history TEXT,
            purchase_history TEXT
        );

        CREATE TABLE IF NOT EXISTS data_order_tokped (
            order_id VARCHAR(20) PRIMARY KEY,
            customer_id VARCHAR(20) REFERENCES data_user_tokped(customer_id),
            product_id VARCHAR(20) REFERENCES data_product_exsport(product_id),
            quantity INT,
            order_date DATE,
            order_status VARCHAR(50),
            shipping_method VARCHAR(50),
            payment_method VARCHAR(50),
            total_price INT
        );

        CREATE TABLE IF NOT EXISTS data_review_order_tokped (
            order_id VARCHAR(20) REFERENCES data_order_tokped(order_id),
            reviewer_id VARCHAR(20) REFERENCES data_user_tokped(customer_id),
            product_id VARCHAR(20) REFERENCES data_product_exsport(product_id),
            review_text TEXT,
            rating FLOAT,
            review_date DATE,
            PRIMARY KEY (order_id, reviewer_id, product_id)
        );
        """)

        conn.commit()
        cursor.close()
        conn.close()
        print("Tables created successfully!")
    
    except Exception as e:
        print(f"Error creating tables: {e}")

# Fungsi untuk mengimpor data dari dataframe ke PostgreSQL
def insert_data(df, table_name):
    try:
        conn = create_connection()
        cursor = conn.cursor()

        # Mengubah dataframe ke format yang sesuai untuk insert
        for i, row in df.iterrows():
            query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(query, tuple(row))
        
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Data inserted into {table_name} successfully!")
    
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Fungsi utama untuk mengimpor data ke dalam PostgreSQL
def main():
    # Contoh memuat data dari CSV (sesuaikan dengan data yang ada pada kamu)
    df_category = pd.read_csv("clean-data/data_category_exsport.csv")
    df_color = pd.read_csv("clean-data/data_color_exsport.csv")
    df_product = pd.read_csv("clean-data/data_product_exsport.csv")
    df_stock = pd.read_csv("clean-data/data_stock_exsport.csv")
    df_user = pd.read_csv("clean-data/data_user_tokped.csv")
    df_order = pd.read_csv("clean-data/data_order_tokped.csv")
    df_review = pd.read_csv("clean-data/data_review_order_tokped.csv")

    # Membuat tabel dan mengimpor data
    create_tables()

    insert_data(df_category, 'data_category_exsport')
    insert_data(df_color, 'data_color_exsport')
    insert_data(df_product, 'data_product_exsport')
    insert_data(df_stock, 'data_stock_exsport')
    insert_data(df_user, 'data_user_tokped')
    insert_data(df_order, 'data_order_tokped')
    insert_data(df_review, 'data_review_order_tokped')

if __name__ == "__main__":
    main()


Connection to PostgreSQL DB successful
Tables created successfully!
Connection to PostgreSQL DB successful
Data inserted into data_category_exsport successfully!
Connection to PostgreSQL DB successful
Error inserting data into data_color_exsport: duplicate key value violates unique constraint "data_color_exsport_pkey"
DETAIL:  Key (color_id)=(ECLR684) already exists.

Connection to PostgreSQL DB successful
Error inserting data into data_product_exsport: insert or update on table "data_product_exsport" violates foreign key constraint "data_product_exsport_color_id_fkey"
DETAIL:  Key (color_id)=(ECLR919) is not present in table "data_color_exsport".

Connection to PostgreSQL DB successful
Error inserting data into data_stock_exsport: insert or update on table "data_stock_exsport" violates foreign key constraint "data_stock_exsport_product_id_fkey"
DETAIL:  Key (product_id)=(ETWS885) is not present in table "data_product_exsport".

Connection to PostgreSQL DB successful
Data inserted into

In [ ]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# Inisialisasi Faker untuk generate data acak
fake = Faker()

# Fungsi untuk memastikan ID unik untuk customer dan order
def generate_customer_id(existing_customer_ids):
    """Generate unique customer_id"""
    while True:
        customer_id = f"CSTM{random.randint(10000, 99999)}"
        if customer_id not in existing_customer_ids:
            existing_customer_ids.add(customer_id)
            return customer_id

def generate_order_id(existing_order_ids):
    """Generate unique order_id"""
    while True:
        order_id = f"ORDT{random.randint(100000, 999999)}"
        if order_id not in existing_order_ids:
            existing_order_ids.add(order_id)
            return order_id

# Generate customer data (hanya generate pelanggan yang belum ada)
def generate_customer_data(num_customers, existing_customer_ids):
    """Generate random customer data"""
    customer_data = []
    for _ in range(num_customers):
        customer_id = generate_customer_id(existing_customer_ids)
        gender = random.choice(['Male', 'Female'])
        full_name = fake.first_name_male() if gender == 'Male' else fake.first_name_female()
        full_name += ' ' + fake.last_name()
        location = fake.city()
        browsing_history = ', '.join([fake.word() for _ in range(5)])
        purchase_history = ', '.join([fake.word() for _ in range(random.randint(1, 5))])
        
        customer_data.append({
            "customer_id": customer_id,
            "name": full_name,
            "age": random.randint(18, 65),
            "gender": gender,
            "location": location,
            "browsing_history": browsing_history,
            "purchase_history": purchase_history
        })
    return customer_data

# Generate order data (hanya generate data order yang belum ada)
def generate_order_data(num_orders, existing_order_ids, existing_customer_ids, existing_product_ids):
    """Generate random order data"""
    order_data = []
    for _ in range(num_orders):
        order_id = generate_order_id(existing_order_ids)
        customer_id = random.choice(list(existing_customer_ids))
        product_id = random.choice(list(existing_product_ids))
        quantity = random.randint(1, 3)
        order_date = fake.date_this_year()
        order_status = 'Pending' if order_date > datetime.now() - timedelta(days=7) else 'Selesai'
        shipping_method = random.choice(['JNE', 'J&T', 'SiCepat'])
        payment_method = random.choice(['Transfer Bank', 'e-Wallet', 'COD'])
        total_price = quantity * random.uniform(50000, 300000)  # Example calculation based on quantity
        
        order_data.append({
            "order_id": order_id,
            "customer_id": customer_id,
            "product_id": product_id,
            "quantity": quantity,
            "order_date": order_date,
            "order_status": order_status,
            "shipping_method": shipping_method,
            "payment_method": payment_method,
            "total_price": total_price
        })
    return order_data

# Load the pre-cleaned data
df_product = pd.read_csv('clean-data/data_product_exsport.csv')  # Data produk yang sudah ada
df_stock = pd.read_csv('clean-data/data_stock_exsport.csv')  # Data stok yang sudah ada
df_category = pd.read_csv('clean-data/data_category_exsport.csv')  # Data kategori yang sudah ada
df_color = pd.read_csv('clean-data/data_color_exsport.csv')  # Data warna yang sudah ada
df_name = pd.read_csv('clean-data/data_name_exsport.csv')  # Data nama produk yang sudah ada

# Extract IDs
existing_product_ids = set(df_product['product_id'].unique())
existing_category_ids = set(df_category['category_id'].unique())
existing_color_ids = set(df_color['color_id'].unique())
existing_customer_ids = set()  # Customer IDs yang sudah ada akan di-generate
existing_order_ids = set()  # Order IDs yang sudah ada akan di-generate

# Generate data untuk customer dan order
customer_data = generate_customer_data(50, existing_customer_ids)  # Misalnya generate 50 pelanggan
order_data = generate_order_data(200, existing_order_ids, existing_customer_ids, existing_product_ids)  # Misalnya generate 200 order

# Convert data menjadi DataFrame untuk pemeriksaan lebih lanjut
df_customers = pd.DataFrame(customer_data)
df_orders = pd.DataFrame(order_data)

# Menampilkan contoh data
print("Data Pelanggan:")
print(df_customers.head())

print("\nData Order:")
print(df_orders.head())

# Simpan ke CSV jika perlu
df_customers.to_csv('generated_data_customers.csv', index=False)
df_orders.to_csv('generated_data_orders.csv', index=False)